In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

In [2]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [3]:
ant_master = pickle.load( open( "prop_master.p", "rb" ) )

In [4]:
ant_master

,Name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,prop-4,285,2,3,0,16,16,1,0,16,...,0.0,0,0.900000,0.625000,1,1,62.500000,1,0.5000,0
1,prop-4,285,2,4,0,23,19,1,0,23,...,0.0,0,0.933333,0.600000,1,1,34.500000,1,0.5000,0
2,prop-4,285,4,6,0,8,13,4,0,8,...,0.0,0,0.913043,0.555556,2,2,8.750000,1,0.5000,0
3,prop-4,285,4,5,2,23,53,4,2,21,...,0.0,0,0.960784,0.555556,1,2,70.750000,1,0.5000,0
4,prop-4,285,2,2,0,11,10,1,0,11,...,0.0,0,0.800000,0.666667,1,1,18.500000,1,0.5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,prop-5,362,3,3,0,8,14,1,0,8,...,0.5,0,0.947368,0.666667,0,0,13.666667,2,0.6667,0
8512,prop-5,362,2,3,0,17,17,1,0,17,...,0.0,0,0.857143,0.625000,1,2,25.000000,1,0.5000,0
8513,prop-5,362,2,3,0,12,15,1,0,12,...,0.0,0,0.857143,0.625000,1,1,21.000000,1,0.5000,1
8514,prop-5,362,2,3,0,15,20,1,0,15,...,0.0,0,0.857143,0.625000,1,1,41.000000,3,1.5000,0


In [5]:
test = ant_master[ant_master['version']==452]
test

,Name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,prop-6,452,2,2,0,5,9,1,1,4,...,0.0,0,0.909091,0.750000,1,1,23.500000,1,0.5000,1
1,prop-6,452,3,1,0,1,3,3,1,0,...,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
2,prop-6,452,13,2,0,8,17,78,0,8,...,0.0,0,0.586207,1.000000,1,3,2.769231,1,0.9231,0
3,prop-6,452,2,2,0,5,7,1,1,4,...,0.0,0,0.909091,0.750000,1,2,7.500000,1,0.5000,0
4,prop-6,452,23,1,0,10,23,253,2,8,...,0.0,0,0.000000,0.255435,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,prop-6,452,7,1,0,5,7,21,3,2,...,0.0,0,0.000000,0.535714,0,0,0.000000,1,1.0000,0
252,prop-6,452,2,1,0,3,2,1,2,1,...,0.0,0,0.000000,0.750000,0,0,0.000000,1,1.0000,0
253,prop-6,452,4,1,0,3,8,0,2,1,...,1.0,0,0.000000,0.500000,0,0,6.000000,1,0.7500,0
254,prop-6,452,6,1,0,6,6,15,4,2,...,0.0,0,0.000000,0.333333,0,0,0.000000,1,1.0000,0


In [6]:
train = ant_master[ant_master['version']!=452]
train

,Name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,prop-4,285,2,3,0,16,16,1,0,16,...,0.0,0,0.900000,0.625000,1,1,62.500000,1,0.5000,0
1,prop-4,285,2,4,0,23,19,1,0,23,...,0.0,0,0.933333,0.600000,1,1,34.500000,1,0.5000,0
2,prop-4,285,4,6,0,8,13,4,0,8,...,0.0,0,0.913043,0.555556,2,2,8.750000,1,0.5000,0
3,prop-4,285,4,5,2,23,53,4,2,21,...,0.0,0,0.960784,0.555556,1,2,70.750000,1,0.5000,0
4,prop-4,285,2,2,0,11,10,1,0,11,...,0.0,0,0.800000,0.666667,1,1,18.500000,1,0.5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,prop-5,362,3,3,0,8,14,1,0,8,...,0.5,0,0.947368,0.666667,0,0,13.666667,2,0.6667,0
8512,prop-5,362,2,3,0,17,17,1,0,17,...,0.0,0,0.857143,0.625000,1,2,25.000000,1,0.5000,0
8513,prop-5,362,2,3,0,12,15,1,0,12,...,0.0,0,0.857143,0.625000,1,1,21.000000,1,0.5000,1
8514,prop-5,362,2,3,0,15,20,1,0,15,...,0.0,0,0.857143,0.625000,1,1,41.000000,3,1.5000,0


In [7]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [8]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,2,3,0,16,16,1,0,16,2,2.000000,127,0.0,0,0.900000,0.625000,1,1,62.500000,1,0.5000
1,2,4,0,23,19,1,0,23,2,1.000000,72,0.0,0,0.933333,0.600000,1,1,34.500000,1,0.5000
2,4,6,0,8,13,4,0,8,1,1.000000,42,0.0,0,0.913043,0.555556,2,2,8.750000,1,0.5000
3,4,5,2,23,53,4,2,21,2,0.833333,289,0.0,0,0.960784,0.555556,1,2,70.750000,1,0.5000
4,2,2,0,11,10,1,0,11,2,2.000000,39,0.0,0,0.800000,0.666667,1,1,18.500000,1,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,3,3,0,8,14,1,0,8,2,0.750000,46,0.5,0,0.947368,0.666667,0,0,13.666667,2,0.6667
8512,2,3,0,17,17,1,0,17,2,2.000000,52,0.0,0,0.857143,0.625000,1,2,25.000000,1,0.5000
8513,2,3,0,12,15,1,0,12,2,2.000000,44,0.0,0,0.857143,0.625000,1,1,21.000000,1,0.5000
8514,2,3,0,15,20,1,0,15,2,1.000000,86,0.0,0,0.857143,0.625000,1,1,41.000000,3,1.5000


In [9]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,2,2,0,5,9,1,1,4,1,2.0,49,0.0,0,0.909091,0.750000,1,1,23.500000,1,0.5000
1,3,1,0,1,3,3,1,0,3,2.0,3,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000
2,13,2,0,8,17,78,0,8,13,2.0,49,0.0,0,0.586207,1.000000,1,3,2.769231,1,0.9231
3,2,2,0,5,7,1,1,4,1,2.0,17,0.0,0,0.909091,0.750000,1,2,7.500000,1,0.5000
4,23,1,0,10,23,253,2,8,23,2.0,23,0.0,0,0.000000,0.255435,0,0,0.000000,1,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,7,1,0,5,7,21,3,2,7,2.0,7,0.0,0,0.000000,0.535714,0,0,0.000000,1,1.0000
252,2,1,0,3,2,1,2,1,2,2.0,2,0.0,0,0.000000,0.750000,0,0,0.000000,1,1.0000
253,4,1,0,3,8,0,2,1,4,0.5,30,1.0,0,0.000000,0.500000,0,0,6.000000,1,0.7500
254,6,1,0,6,6,15,4,2,6,2.0,6,0.0,0,0.000000,0.333333,0,0,0.000000,1,1.0000


In [10]:
y_train = train['bug']
y_test = test['bug']

In [11]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
8511    0
8512    0
8513    1
8514    0
8515    8
Name: bug, Length: 69397, dtype: int64

In [12]:
y_test

0      1
1      0
2      0
3      0
4      0
      ..
251    0
252    0
253    0
254    0
255    0
Name: bug, Length: 256, dtype: int64

In [13]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [14]:
x_train_np

array([[ 2.   ,  3.   ,  0.   , ..., 62.5  ,  1.   ,  0.5  ],
       [ 2.   ,  4.   ,  0.   , ..., 34.5  ,  1.   ,  0.5  ],
       [ 4.   ,  6.   ,  0.   , ...,  8.75 ,  1.   ,  0.5  ],
       ...,
       [ 2.   ,  3.   ,  0.   , ..., 21.   ,  1.   ,  0.5  ],
       [ 2.   ,  3.   ,  0.   , ..., 41.   ,  3.   ,  1.5  ],
       [40.   ,  1.   ,  0.   , ...,  8.425,  3.   ,  1.05 ]])

In [15]:
x_test_np

array([[ 2.        ,  2.        ,  0.        , ..., 23.5       ,
         1.        ,  0.5       ],
       [ 3.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [13.        ,  2.        ,  0.        , ...,  2.76923077,
         1.        ,  0.9231    ],
       ...,
       [ 4.        ,  1.        ,  0.        , ...,  6.        ,
         1.        ,  0.75      ],
       [ 6.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 2.        ,  2.        ,  0.        , ...,  4.5       ,
         1.        ,  0.5       ]])

In [16]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [17]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [18]:
train_x

array([[[ 2.   ,  3.   ,  0.   , ..., 62.5  ,  1.   ,  0.5  ]],

       [[ 2.   ,  4.   ,  0.   , ..., 34.5  ,  1.   ,  0.5  ]],

       [[ 4.   ,  6.   ,  0.   , ...,  8.75 ,  1.   ,  0.5  ]],

       ...,

       [[ 2.   ,  3.   ,  0.   , ..., 21.   ,  1.   ,  0.5  ]],

       [[ 2.   ,  3.   ,  0.   , ..., 41.   ,  3.   ,  1.5  ]],

       [[40.   ,  1.   ,  0.   , ...,  8.425,  3.   ,  1.05 ]]])

In [19]:
test_y

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])

Epoch 1/100
694/694 [==============================] - 8s 6ms/step - loss: 0.6689 - mse: 0.6689 - mae: 0.3292 - root_mean_squared_error: 0.8179 - mean_squared_logarithmic_error: 0.1056
Epoch 2/100
694/694 [==============================] - 4s 6ms/step - loss: 0.6167 - mse: 0.6167 - mae: 0.3237 - root_mean_squared_error: 0.7853 - mean_squared_logarithmic_error: 0.1005
Epoch 3/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5996 - mse: 0.5996 - mae: 0.3163 - root_mean_squared_error: 0.7743 - mean_squared_logarithmic_error: 0.0990
Epoch 4/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5935 - mse: 0.5935 - mae: 0.3156 - root_mean_squared_error: 0.7704 - mean_squared_logarithmic_error: 0.0981
Epoch 5/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5876 - mse: 0.5876 - mae: 0.3152 - root_mean_squared_error: 0.7665 - mean_squared_logarithmic_error: 0.0976
Epoch 6/100
694/694 [==============================] - 4s 6ms/step - loss: 

694/694 [==============================] - 5s 7ms/step - loss: 0.5598 - mse: 0.5598 - mae: 0.3105 - root_mean_squared_error: 0.7482 - mean_squared_logarithmic_error: 0.0942
Epoch 45/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5590 - mse: 0.5590 - mae: 0.3102 - root_mean_squared_error: 0.7477 - mean_squared_logarithmic_error: 0.0945
Epoch 46/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5649 - mse: 0.5649 - mae: 0.3118 - root_mean_squared_error: 0.7516 - mean_squared_logarithmic_error: 0.0948
Epoch 47/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5603 - mse: 0.5603 - mae: 0.3113 - root_mean_squared_error: 0.7486 - mean_squared_logarithmic_error: 0.0946
Epoch 48/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5581 - mse: 0.5581 - mae: 0.3108 - root_mean_squared_error: 0.7470 - mean_squared_logarithmic_error: 0.0943
Epoch 49/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5582 

694/694 [==============================] - 4s 6ms/step - loss: 0.5505 - mse: 0.5505 - mae: 0.3093 - root_mean_squared_error: 0.7420 - mean_squared_logarithmic_error: 0.0932
Epoch 89/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5431 - mse: 0.5431 - mae: 0.3090 - root_mean_squared_error: 0.7370 - mean_squared_logarithmic_error: 0.0930
Epoch 90/100
694/694 [==============================] - 5s 7ms/step - loss: 0.5488 - mse: 0.5488 - mae: 0.3092 - root_mean_squared_error: 0.7408 - mean_squared_logarithmic_error: 0.0932
Epoch 91/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5456 - mse: 0.5456 - mae: 0.3080 - root_mean_squared_error: 0.7387 - mean_squared_logarithmic_error: 0.0930
Epoch 92/100
694/694 [==============================] - 5s 7ms/step - loss: 0.5487 - mse: 0.5487 - mae: 0.3087 - root_mean_squared_error: 0.7407 - mean_squared_logarithmic_error: 0.0931
Epoch 93/100
694/694 [==============================] - 4s 6ms/step - loss: 0.5443 

In [22]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 0:02:20 ago. (Use '!kill 3750' to kill it.)

In [23]:
score = model.evaluate(test_x, test_y)
score

8/8 [==============================] - 1s 3ms/step - loss: 0.1243 - mse: 0.1243 - mae: 0.2124 - root_mean_squared_error: 0.3525 - mean_squared_logarithmic_error: 0.0551


[0.1242629885673523,
 0.1242629885673523,
 0.21243593096733093,
 0.3525095582008362,
 0.05514729022979736]